# وكلاء Azure AI مع دعم بروتوكول سياق النموذج (MCP)

يوضح هذا الدفتر كيفية استخدام وكلاء Azure AI مع أدوات بروتوكول سياق النموذج (MCP) لإنشاء وكيل ذكي يمكنه الاستفادة من خوادم MCP الخارجية لتعزيز قدراته.


## تثبيت حزم NuGet المطلوبة

أولاً، نحتاج إلى تثبيت حزمة Azure AI Agents Persistent التي توفر الوظائف الأساسية للعمل مع Azure AI Agents.


## فوائد المصادقة بدون مفاتيح

يوضح هذا الدفتر فوائد **المصادقة بدون مفاتيح** التي توفر العديد من المزايا:
- ✅ **لا حاجة لإدارة مفاتيح API** - تستخدم المصادقة المستندة إلى هوية Azure
- ✅ **أمان معزز** - لا يتم تخزين أي أسرار في الكود أو الإعدادات
- ✅ **تدوير تلقائي للاعتمادات** - تتولى Azure إدارة دورة حياة الاعتمادات
- ✅ **الوصول المستند إلى الأدوار** - تستخدم Azure RBAC لتحديد الأذونات بدقة

سيقوم `DefaultAzureCredential` تلقائيًا باستخدام أفضل مصدر اعتماد متاح:
1. الهوية المُدارة (عند التشغيل في Azure)
2. اعتمادات Azure CLI (أثناء التطوير)
3. اعتمادات Visual Studio
4. متغيرات البيئة (إذا تم تكوينها)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

قم بتثبيت حزمة Azure Identity للمصادقة مع خدمات Azure باستخدام DefaultAzureCredential.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## استيراد المساحات المطلوبة

قم باستيراد المساحات اللازمة لوكلاء Azure AI وهوية Azure.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## إعداد عميل Azure AI Agent (المصادقة بدون مفتاح)

قم بتكوين المتغيرات وإنشاء PersistentAgentsClient باستخدام **المصادقة بدون مفتاح**:
- **projectEndpoint**: نقطة النهاية لمشروع Azure AI Foundry
- **modelDeploymentName**: اسم النموذج الذكي المنشور (GPT-4.1 nano)
- **mcpServerUrl**: عنوان URL لخادم MCP (واجهة برمجة تطبيقات Microsoft Learn)
- **mcpServerLabel**: تسمية لتحديد خادم MCP
- **DefaultAzureCredential**: يستخدم الهوية المُدارة، Azure CLI، أو مصادر بيانات اعتماد أخرى (لا حاجة لمفاتيح API)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## إنشاء تعريف أداة MCP

قم بإنشاء تعريف أداة MCP يتصل بخادم Microsoft Learn MCP. سيسمح ذلك للوكيل بالوصول إلى محتوى Microsoft Learn والوثائق.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## إنشاء الوكيل الذكي

قم بإنشاء وكيل ذكي دائم باستخدام النموذج المحدد وأدوات MCP. يتم تكوين الوكيل باستخدام:
- نموذج GPT-4.1 nano  
- تعليمات لاستخدام أدوات MCP للمساعدة  
- الوصول إلى خادم Microsoft Learn MCP  


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## إنشاء سلسلة محادثة وإرسال رسالة

قم بإنشاء سلسلة محادثة وأرسل رسالة للمستخدم تسأله عن الفرق بين Azure OpenAI و OpenAI. سيختبر هذا قدرة الوكيل على استخدام أدوات MCP لتقديم معلومات دقيقة.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## إعداد موارد أداة MCP (بدون مفاتيح)

قم بإعداد موارد أداة MCP. للحصول على نهج خالٍ تمامًا من المفاتيح، يمكنك إزالة الرؤوس المخصصة إذا كان خادم MCP يدعم المصادقة المستندة إلى هوية Azure. يوضح المثال أدناه كيفية إضافة الرؤوس إذا لزم الأمر، ولكن في السيناريوهات الخالية من المفاتيح، قد لا تكون هذه مطلوبة.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## بدء تشغيل الوكيل

قم بإنشاء وتشغيل عملية لمعالجة رسالة المستخدم. سيستخدم الوكيل الأدوات والموارد المهيأة لـ MCP لتوليد استجابة.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## مراقبة تشغيل الوكيل ومعالجة موافقات الأدوات (بدون مفتاح)

راقب حالة تشغيل الوكيل وتعامل مع أي موافقات مطلوبة للأدوات. هذه الدورة:
1. تنتظر اكتمال التشغيل أو الحاجة إلى إجراء
2. توافق تلقائيًا على استدعاءات أدوات MCP عند الحاجة
3. بالنسبة للمصادقة بدون مفتاح، قد لا تكون الرؤوس مطلوبة إذا كان خادم MCP يدعم هوية Azure


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## عرض نتائج المحادثة

استرجاع وعرض جميع الرسائل في سلسلة المحادثة، مع إظهار كل من سؤال المستخدم ورد الوكيل. يتم عرض الرسائل بترتيب زمني مع تضمين الطوابع الزمنية ومؤشرات الأدوار.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**إخلاء المسؤولية**:  
تم ترجمة هذا المستند باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو معلومات غير دقيقة. يجب اعتبار المستند الأصلي بلغته الأصلية المصدر الرسمي. للحصول على معلومات حاسمة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة ناتجة عن استخدام هذه الترجمة.
